# data

> All the utility code related to data loading and processing

In [ ]:
#| default_exp data.core

In [ ]:
#| include: false
from nbdev.showdoc import *

In [ ]:
#| export 
import torch
from torch.utils.data import TensorDataset
from fastai.tabular.all import *
from tsai.all import *

In [ ]:
#| export 
def create_dls(x, y, bs, is_rest=True):
    # Transfer Learning to BIOWIN data
    # # Normalize input data
    eps = 1e-08
    mean = torch.nanmean(x, dim=2)
    std = torch.clamp_min(torch.std(x, dim=2), eps)
    x_norm = torch.stack([torch.vstack([(x[j,i,:]-mean[j,i])/torch.clamp_min(std[j,i], eps)
                        for i in range(x.shape[1])]) 
                        for j in range(x.shape[0])],dim=0)
    x_norm = torch.clamp_min(x_norm,-3)
    x_norm = torch.clamp_max(x_norm,3).float()

    # Load the labels
    if is_rest: 
        y_tensor = torch.tensor([[0, 1] if lab == 0 else [1, 0] for lab in y], device=device).float()
    else: 
        y_tensor = torch.tensor([[0, 1] if lab == 1 else [1, 0] for lab in y], device=device).float()

    # Create a TensorDataset
    ds = TensorDataset(x_norm, y_tensor)

    # Split your dataset into training and validation sets
    split = RandomSplitter(valid_pct=0.2, seed=42)(ds)
    train_ds = torch.utils.data.Subset(ds, split[0])
    valid_ds = torch.utils.data.Subset(ds, split[1])

    # Create DataLoaders
    dls = DataLoaders.from_dsets(train_ds, valid_ds, bs=bs, num_workers=0)

    return dls

In [ ]:
show_doc(create_dls)

---

### create_dls

>      create_dls (x, y, bs, is_rest=True)

## Test the dls creation
...

In [ ]:
#| export
def create_dls_eoec(x, y, bs):
    n_train_samples = int(x.shape[0]//4*3) # training set = 80% of the dataset
    n_total_samples = x.shape[0]
    splits = (L(range(n_train_samples), use_list=True),
            L(np.arange(n_train_samples, n_total_samples), use_list=True))
    splitter = IndexSplitter(splits[1])

    # Create a DataLoaders object for the EEG data in input and output for the autoencoder
    getters = [ItemGetter(0), ItemGetter(1)]
    catblock = MultiCategoryBlock(encoded=True, vocab=['Eyes Closed', 'Eyes Open'])
    dblock = DataBlock(blocks=(TSTensorBlock, catblock),
                        splitter=splitter,
                        getters=getters,
                        batch_tfms=norm_batch())
    src = itemify(x,y)
    dls = dblock.dataloaders(src, bs=bs, drop_last=True)

    return dls

In [ ]:
show_doc(create_dls_eoec)

---

### create_dls_eoec

>      create_dls_eoec (x, y, bs)